# Scenario 3b: Multiple data scientists working on multiple ML models (with terraform)

MLflow setup:

- Tracking server: yes, remote server (GCP VM).
- Backend store: postgresql database.
- Artifacts store: GC bucket.
The experiments can be explored by accessing the remote server.

The exampe uses GCP to host a remote server. In order to run the example you'll need an GCP account.

In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/home/habeeb/dprof-dezoomfinal-b4d188529d18.json"

In [2]:
import mlflow
import os

TRACKING_SERVER_HOST = "35.224.212.79" # fill in with the public IP
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment("my-gcp-experiment")

2025/06/04 12:15:49 INFO mlflow.tracking.fluent: Experiment with name 'my-gcp-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://dezoomfinal-mlflow-artifacts/1', creation_time=1749060949843, experiment_id='1', last_update_time=1749060949843, lifecycle_stage='active', name='my-gcp-experiment', tags={}>

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

#mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/06/04 12:16:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'gs://dezoomfinal-mlflow-artifacts/1/7bf08e50f170410b80f4237d2a95c83d/artifacts'
🏃 View run luxuriant-seal-991 at: http://35.224.212.79:5000/#/experiments/1/runs/7bf08e50f170410b80f4237d2a95c83d
🧪 View experiment at: http://35.224.212.79:5000/#/experiments/1


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='gs://mlfflow-bucket-nyc/1', creation_time=1747605570875, experiment_id='1', last_update_time=1747605570875, lifecycle_stage='active', name='my-gcp-experiment', tags={}>,
 <Experiment: artifact_location='gs://mlfflow-bucket-nyc/0', creation_time=1747603153225, experiment_id='0', last_update_time=1747603153225, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
client.search_registered_models()

[]

In [8]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids=["1"],
    filter_string="",
    order_by=["metrics.rmse ASC"],
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5
)
print(runs)

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "697fabbc2d02471090e6db6038926565", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2025-05-18 22:13:22.454335", "model_uuid": '
                             '"dd3a3a3ae92d4f27ba23a97049bb93b7", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.7", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.1", '
                             '"serialization_format": "cloudpickle", "code": '
             

In [9]:
first_run_id = runs[0].info.run_id

In [10]:
mlflow.register_model(
    model_uri=f'runs:/{first_run_id}/models',
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/18 16:15:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1747606551812, current_stage='None', description='', last_updated_timestamp=1747606551812, name='iris-classifier', run_id='697fabbc2d02471090e6db6038926565', run_link='', source='gs://mlfflow-bucket-nyc/1/697fabbc2d02471090e6db6038926565/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>